# EDA 1 | Parte 3 - Análise dos Textos dos Hinos da ICM
Este notebook explora o conteúdo textual dos hinos da coletânea principal da Igreja Cristã Maranata.

O objetivo é analisar o texto dos louvores, extraindo informações como número de palavras, tokens, frequências, n-grams e similaridade entre hinos.

---
**Conteúdo do notebook:**
- Carregamento dos dados tratados
- Tokenização e remoção de stopwords
- Análise do número de palavras por hino
- Visualização da distribuição de tokens por categoria
- Análise de palavras mais longas e frequentes
- Geração de nuvem de palavras
- Extração de n-grams (bigramas, trigramas)
- Cálculo de similaridade entre hinos (CountVectorizer e TF-IDF)

---

**A seguir:** Importação e preparação da lista de stopwords, combinando fontes externas e do NLTK para uso na tokenização.

In [ ]:
import pandas as pd
from pathlib import Path

assets_folder = Path("../assets")
hinos_analise: pd.DataFrame = pd.read_pickle(assets_folder / "hinos_analise.pkl")
hinos_analise = hinos_analise.set_index("numero")
hinos_analise["categoria_abr"] = hinos_analise["categoria"].apply(
    lambda x: x[:13] + "..." if len(x) > 15 else x
)
hinos_analise

---

## Tokenização

**A seguir:** Tokenização dos textos dos hinos, remoção de stopwords e pontuação, e cálculo do número total de palavras por hino.

In [ ]:
import nltk

nltk.download("stopwords")
stopwords_nltk = nltk.corpus.stopwords.words("portuguese")

with open(assets_folder / "stopwords-br.txt", "r", encoding="utf-8") as f:
    stopwords = f.read().splitlines()

# remover linhas que comecao com #
stopwords = [eval(word) for word in stopwords if not word.startswith("#")]
stopwords.extend(["ó", "ti", "pra", "lo", "oh", "és"])

# merge
stopwords = list(set(stopwords + stopwords_nltk))
len(stopwords)

---

**A seguir:** Exibição dos 10 hinos com maior e menor quantidade de palavras, destacando extremos do corpus.

In [ ]:
from tqdm import tqdm


all_tokens = []
all_tokens_no_stops = []

for hino in tqdm(hinos_analise.to_dict("records")):
    tokens = nltk.tokenize.regexp_tokenize(hino["texto_limpo"], r"\w+")
    # Replace "MINH" com "MINHA" usando regex
    tokens = [nltk.re.sub(r"^minh$", "minha", palavra.lower()) for palavra in tokens]
    tokens_no_stops = [
        palavra for palavra in tokens if palavra.lower() not in stopwords
    ]
    # remover pontuacao
    tokens = [palavra for palavra in tokens if palavra.isalpha()]
    tokens_no_stops = [palavra for palavra in tokens_no_stops if palavra.isalpha()]

    all_tokens.append(tokens)
    all_tokens_no_stops.append(tokens_no_stops)

hinos_analise["tokens"] = all_tokens
hinos_analise["tokens_no_stops"] = all_tokens_no_stops
# considerando numero total de palavras, pois todas elas tem que ser cantadas, logo impactam no tamanho prático do hino
hinos_analise["num_tokens"] = hinos_analise["tokens"].apply(len)
hinos_analise

In [ ]:
display(hinos_analise.sort_values("num_tokens", ascending=False).head(10))
display(hinos_analise.sort_values("num_tokens", ascending=True).head(10))

---

**A seguir:** Visualização da distribuição do número de palavras por categoria, utilizando boxplot para identificar padrões e variações.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure 'categoria_id' is treated as a categorical variable
hinos_analise["categoria_id"] = hinos_analise["categoria_id"].astype("category")

# Create a mapping between categoria_id and categoria
categoria_mapping = (
    hinos_analise[["categoria_id", "categoria_abr"]]
    .drop_duplicates()
    .set_index("categoria_id")["categoria_abr"]
)

# Create a Boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=hinos_analise, x="categoria_id", y="num_tokens", palette="viridis")

# Replace x-ticks with corresponding 'categoria' names
plt.xticks(
    ticks=range(len(categoria_mapping)),
    labels=categoria_mapping,
    rotation=90,
    ha="right",
)

# Add labels and title
plt.xlabel("Categoria")
plt.ylabel("Number of Tokens")
plt.title("Relationship Between Number of Tokens and Categoria (Box Plot)")

# Show the plot
plt.tight_layout()
plt.show()

---

**A seguir:** Análise de palavras: extração, identificação das mais longas, contagem de frequência e visualização das palavras mais comuns com nuvem de palavras e gráfico de barras.

## Histograma de frequência de tamanho das palavras

**A seguir:** Geração de um histograma mostrando a frequência dos tamanhos das palavras nos textos dos hinos, para entender a distribuição do comprimento das palavras utilizadas.

In [ ]:
# Make a frequency list of lengths: line_num_words
line_num_words = [
    len(t_line) for t_line in hinos_analise["tokens_no_stops"].explode().tolist()
]

# Plot a histogram of the line lengths
plt.hist(line_num_words)

# Show the plot
plt.show()

## Palavras mais longas

**A seguir:** Exibição das 10 palavras mais longas encontradas nos textos dos hinos, destacando a diversidade lexical do corpus.

In [ ]:
palavras = hinos_analise["tokens_no_stops"].explode().tolist()

# find the 10 largest words
palavras_unique = list(set(palavras))
palavras_unique.sort(key=len, reverse=True)
print("Quantidade de palavras únicas (sem repetição):",len(palavras_unique))
pd.DataFrame(
    {
        "palavra": palavras_unique[:10],
        "tamanho": [len(palavra) for palavra in palavras_unique[:10]],
    }
)

## Bag-of-words

**A seguir:** Criação de uma representação bag-of-words dos textos dos hinos, incluindo a frequência de cada palavra e um mapeamento das palavras para seus índices.

In [ ]:
print("Total de palavras:",len(palavras))
set_words_full = list(set(palavras))
count_words = [palavras.count(i) for i in set_words_full]

contagem_palav = pd.DataFrame(
    zip(set_words_full, count_words), columns=["palavra", "contagem"]
)
contagem_palav = contagem_palav.sort_values("contagem", ascending=False)
contagem_palav["percentual"] = contagem_palav["contagem"] / len(palavras) * 100
contagem_palav

In [ ]:
import matplotlib.pyplot as plt

# Also show top 20 most frequent words as a bar chart
plt.figure(figsize=(12, 8))
top_20 = contagem_palav.head(20)
plt.barh(range(len(top_20)), top_20["contagem"], color="skyblue")
plt.yticks(range(len(top_20)), top_20["palavra"])
plt.xlabel("Frequência")
plt.title("Top 20 Palavras Mais Frequentes")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Word cloud

**A seguir:** Geração de uma nuvem de palavras para visualizar as palavras mais frequentes nos textos dos hinos, destacando termos recorrentes e temas predominantes.

In [ ]:
from wordcloud import WordCloud

# Create a dictionary from the word frequency data
word_freq_dict = dict(zip(contagem_palav["palavra"], contagem_palav["contagem"]))

# Generate word cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color="white",
    max_words=100,
    colormap="viridis",
    relative_scaling=0.5,
    random_state=42,
).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud - Palavras mais frequentes nos hinos", fontsize=16, pad=20)
plt.tight_layout()
plt.show()

## N-grams

---

**A seguir:** Extração e análise de n-grams (bigramas e trigramas) para identificar padrões de palavras recorrentes nos textos dos hinos.

In [ ]:
from collections import Counter


# Exemplo: gerar bigramas do corpus inteiro
def get_bigrams(tokens):
    return list(nltk.ngrams(tokens, 2))  # 2 = bigramas


# Gerar bigramas para todos os hinos
hinos_analise["bigrams"] = hinos_analise["tokens_no_stops"].apply(get_bigrams)

# Contar bigramas mais frequentes no corpus inteiro
all_bigrams = [bigram for hino in hinos_analise["bigrams"] for bigram in hino]
bigram_freq = Counter(all_bigrams)

bigram_freq.most_common(10)

In [ ]:
# Gerar trigramas do corpus inteiro
def get_trigrams(tokens):
    return list(nltk.ngrams(tokens, 3))  # 3 = trigrams


# Gerar trigrams para todos os hinos
hinos_analise["trigrams"] = hinos_analise["tokens_no_stops"].apply(get_trigrams)

# Contar trigrams mais frequentes no corpus inteiro
all_trigrams = [trigram for hino in hinos_analise["trigrams"] for trigram in hino]
trigram_freq = Counter(all_trigrams)

trigram_freq.most_common(10)

## Matriz de frequência e similaridade CountVectorizer

---

**A seguir:** Cálculo de matriz de similaridade entre hinos usando CountVectorizer, visualização com heatmap e identificação de hinos similares por esse método.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Juntar os tokens em strings
hinos_analise["tokens_str"] = hinos_analise["tokens_no_stops"].apply(
    lambda t: " ".join(t)
)

# Criar o vetor de frequências
vectorizer = CountVectorizer(
    ngram_range=(1, 3), stop_words=None
)  # unigramas, bigramas e trigramas
X = vectorizer.fit_transform(hinos_analise["tokens_str"])

# Similaridade de cosseno entre hinos
similarity = cosine_similarity(X)
similarity_df = pd.DataFrame(
    similarity, index=hinos_analise.index, columns=hinos_analise.index
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 10))
sns.heatmap(similarity_df, cmap="viridis", annot=False)
plt.title("Similaridade entre hinos")
plt.show()

In [ ]:
high_similarity = similarity_df[
    (similarity_df > 0.5) & (similarity_df < 1.0)
].stack()  # .reset_index()
high_similarity = high_similarity[
    high_similarity.index.get_level_values(0)
    < high_similarity.index.get_level_values(1)
]
high_similarity.sort_values(ascending=False).head()

## Matriz com TF-IDF

**A seguir:** Cálculo de matriz de similaridade entre hinos usando TF-IDF, visualização com heatmap e identificação de hinos similares por esse método.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF: unigrams e bigrams
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=None)
X_tfidf = vectorizer.fit_transform(hinos_analise["tokens_str"])

**A seguir:** Ranking de termos mais relevantes por hino com TF-IDF, destacando palavras-chave que caracterizam cada hino individualmente (uni e bigramas).

In [ ]:
import random

def top_terms_for_hymn(row, features, top_n=5):
    row_data = list(zip(features, row))
    row_data = sorted(row_data, key=lambda x: x[1], reverse=True)
    return row_data[:top_n]


features = vectorizer.get_feature_names_out()

# random.seed(42)
sample_idxs = random.sample(range(X_tfidf.shape[0]), 3)

for idx in sample_idxs:
    row = X_tfidf[idx].toarray().ravel()
    top_terms = top_terms_for_hymn(row, features, top_n=3)
    hymn_num = hinos_analise.index[idx]
    hymn_name = hinos_analise.loc[hymn_num, "nome"]
    print(f"\n🎵 Hino {hymn_num} — {hymn_name}:")
    for term, score in top_terms:
        print(f"  {term}: {score:.3f}")

**A seguir:** Construindo a matriz de similaridade entre hinos usando TF-IDF

In [ ]:
similarity_tfidf = cosine_similarity(X_tfidf)
similarity_df_tfidf = pd.DataFrame(
    similarity_tfidf, index=hinos_analise.index, columns=hinos_analise.index
)

**A seguir:** Visualização com heatmap e identificação de hinos similares por esse método.

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_df_tfidf, cmap="viridis", annot=False)
plt.title("Similaridade entre hinos (TF-IDF)")
plt.show()

In [ ]:
high_similarity_tfidf = similarity_df_tfidf[
    (similarity_df_tfidf > 0.5) & (similarity_df_tfidf < 1.0)
].stack()  # .reset_index()
high_similarity_tfidf = high_similarity_tfidf[
    high_similarity_tfidf.index.get_level_values(0)
    < high_similarity_tfidf.index.get_level_values(1)
]
high_similarity_tfidf.sort_values(ascending=False).head(10)

---

**A seguir:** Exportação dos dados tratados e enriquecidos para arquivo pickle, permitindo reutilização em outras análises ou notebooks.

In [ ]:
hinos_analise.to_pickle(assets_folder / "hinos_analise_tokens.pkl")